In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
train_ds, test_ds = tfds.load('imdb_reviews', split=['train', 'test'], as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteTKPM0Y/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteTKPM0Y/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteTKPM0Y/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [7]:
train_data = []
train_labels = []

for example, label in train_ds:
  train_data.append(example.numpy().decode('utf-8'))
  train_labels.append(label.numpy())

test_data = []
test_labels = []

for example, label in test_ds:
  test_data.append(example.numpy().decode('utf-8'))
  test_labels.append(label.numpy())


train_data = np.array(train_data)
train_labels = np.array(train_labels)
test_data = np.array(test_data)
test_labels = np.array(test_labels)

In [8]:
tokenizer = Tokenizer(num_words=10000, oov_token='<OOV>')
tokenizer.fit_on_texts(train_data)
word_index = tokenizer.word_index
total_words = len(word_index) + 1
train_sequences = tokenizer.texts_to_sequences(train_data)

In [11]:
train_sequences = pad_sequences(train_sequences,
                                maxlen=256,
                                padding='post')

In [13]:
class MyCallback(tf.keras.callbacks.Callback):
  def on_epochs_end(self, epoch, logs={}):
    if logs.get('accuracy') > 0.95:
      self.model.stop_training = True

callback = MyCallback()

In [15]:
model = tf.keras.Sequential([
  tf.keras.layers.Embedding(10000, 32, input_length=256),
  tf.keras.layers.Bidirectional(
      tf.keras.layers.LSTM(64, return_sequences=True)
  ),
    tf.keras.layers.Bidirectional(
      tf.keras.layers.LSTM(64, return_sequences=True)
  ),
    tf.keras.layers.Bidirectional(
      tf.keras.layers.LSTM(64, return_sequences=True)
  ),
  tf.keras.layers.LSTM(64),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(32, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid'),
])

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(train_sequences, train_labels,
                    epochs=10,
                    callbacks=[callback])

Epoch 1/10
782/782 [==============================] - 827s 1s/step - loss: 0.6909 - accuracy: 0.5133
Epoch 2/10
782/782 [==============================] - 817s 1s/step - loss: 0.6823 - accuracy: 0.5203
Epoch 3/10
782/782 [==============================] - 817s 1s/step - loss: 0.6916 - accuracy: 0.5034
Epoch 4/10
782/782 [==============================] - 821s 1s/step - loss: 0.6916 - accuracy: 0.5036
Epoch 5/10
782/782 [==============================] - 819s 1s/step - loss: 0.6918 - accuracy: 0.5031
Epoch 6/10
782/782 [==============================] - 819s 1s/step - loss: 0.6918 - accuracy: 0.4984
Epoch 7/10
782/782 [==============================] - 820s 1s/step - loss: 0.6917 - accuracy: 0.4990
Epoch 8/10
782/782 [==============================] - 820s 1s/step - loss: 0.6917 - accuracy: 0.5011
Epoch 9/10
782/782 [==============================] - 818s 1s/step - loss: 0.6916 - accuracy: 0.5015
Epoch 10/10
782/782 [==============================] - 818s 1s/step - loss: 0.6919 - accura

In [16]:
def predict(model, data, labels):

  i = random.randint(0, 10000)


  prediction = model.predict(tf.reshape(data[i], (1, 32, 32 ,3)))

  print('Prediction: ', prediction.argmax())
  print('Label: ', labels[i])

  if prediction.argmax() - labels[i] == 0:
    print('Correct')
    print()
  else:
    print('Incorrect')
    print()

In [17]:
for i in range(25):
  predict(model=model, data=test_data, labels=test_labels)

UnimplementedError: ignored